Simulating multi sensor data

In [1]:
import numpy as np
import pandas as pd

def generate_smart_failure_data(minutes=500):
    np.random.seed(42)
    temp = [25]
    hum = [40]
    vib = [0.02]
    power = [1.0]

    failure_risk = []

    for i in range(minutes):
        # Evolve sensors
        t = temp[-1] + np.random.normal(0, 0.5)
        h = hum[-1] + np.random.normal(0, 1)
        v = max(0, vib[-1] + np.random.normal(0, 0.01))
        p = power[-1] + np.random.normal(0, 0.02)

        # Append
        temp.append(round(t, 2))
        hum.append(round(h, 2))
        vib.append(round(v, 3))
        power.append(round(p, 3))

        # Define failure RISK probabilistically (not with thresholds)
        # Higher risk if multiple sensors are slightly abnormal
        prob_failure = (
            0.05 +                # base risk
            0.01 * max(0, t - 27) +
            0.01 * max(0, h - 50) +
            0.5  * (v > 0.05) +   # spikes in vibration matter more
            0.2  * (p > 1.1)
        )
        risk = np.random.rand() < prob_failure  # draw 1 with calculated probability
        failure_risk.append(int(risk))

    df = pd.DataFrame({
        'temperature': temp[1:],
        'humidity': hum[1:],
        'vibration': vib[1:],
        'power_drift': power[1:],
        'failure_risk': failure_risk
    })

    df['temp_diff'] = df['temperature'].diff().fillna(0)
    df['vib_diff'] = df['vibration'].diff().fillna(0)
    return df


Model training

In [2]:
df = generate_smart_failure_data()

features = ['temperature', 'humidity', 'vibration', 'power_drift', 'temp_diff', 'vib_diff']
X = df[features]
y = df['failure_risk']

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier()
model.fit(X_train, y_train)

print(f"📈 Model trained. Accuracy: {model.score(X_test, y_test):.2f}")


📈 Model trained. Accuracy: 0.87


Real time prediction

In [4]:
import time
import numpy as np
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

# Simulate one reading step with realistic anomalies
def simulate_sensor_reading(prev_temp, prev_hum, prev_vib, prev_power, step):
    # Inject spikes every 4 steps (~2 mins)
    spike = (step % 4 == 0 and step != 0)

    temp = prev_temp + (np.random.normal(0, 0.5) + (5 if spike else 0))
    hum = prev_hum + (np.random.normal(0, 1.0) + (10 if spike else 0))
    vib = max(0, prev_vib + np.random.normal(0, 0.01) + (0.3 if spike else 0))
    power = prev_power + np.random.normal(0, 0.02) + (0.3 if spike else 0)

    temp_diff = temp - prev_temp
    vib_diff = vib - prev_vib

    return round(temp, 2), round(hum, 2), round(vib, 3), round(power, 3), round(temp_diff, 2), round(vib_diff, 3)

# Initialize values
current_temp = 26.0
current_hum = 45.0
current_vib = 0.02
current_power = 1.0

print("🔁 Starting real-time smart failure prediction...\n")

# Run for 7 minutes (14 readings)
for step in range(14):
    # Generate new data point
    t, h, v, p, td, vd = simulate_sensor_reading(current_temp, current_hum, current_vib, current_power, step)

    # Update for next iteration
    current_temp, current_hum, current_vib, current_power = t, h, v, p

    # Predict failure risk (make sure model is trained already)
    input_data = [[t, h, v, p, td, vd]]
    prediction = model.predict(input_data)[0]  # model should be trained using previous cell

    # Display result
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] Temp: {t:.2f}°C | Hum: {h:.1f}% | Vib: {v:.3f}g | Power Drift: {p:.3f} | ", end="")
    if prediction == 1:
        print("⚠️  FAILURE RISK detected!")
    else:
        print("✅  Status: Normal.")

    time.sleep(2)  # Use 30 for actual real-time pacing


🔁 Starting real-time smart failure prediction...

[16:54:49] Temp: 25.83°C | Hum: 44.4% | Vib: 0.009g | Power Drift: 0.983 | ✅  Status: Normal.
[16:54:51] Temp: 27.30°C | Hum: 45.6% | Vib: 0.000g | Power Drift: 0.957 | ✅  Status: Normal.
[16:54:53] Temp: 27.54°C | Hum: 46.2% | Vib: 0.005g | Power Drift: 0.952 | ✅  Status: Normal.
[16:54:55] Temp: 27.48°C | Hum: 46.5% | Vib: 0.006g | Power Drift: 0.908 | ✅  Status: Normal.
[16:54:57] Temp: 32.37°C | Hum: 55.7% | Vib: 0.308g | Power Drift: 1.268 | ⚠️  FAILURE RISK detected!
[16:54:59] Temp: 32.55°C | Hum: 55.4% | Vib: 0.317g | Power Drift: 1.278 | ✅  Status: Normal.
[16:55:01] Temp: 32.76°C | Hum: 56.0% | Vib: 0.338g | Power Drift: 1.255 | ⚠️  FAILURE RISK detected!
[16:55:03] Temp: 33.00°C | Hum: 55.0% | Vib: 0.344g | Power Drift: 1.237 | ⚠️  FAILURE RISK detected!
[16:55:05] Temp: 37.79°C | Hum: 64.7% | Vib: 0.654g | Power Drift: 1.555 | ⚠️  FAILURE RISK detected!
[16:55:07] Temp: 37.17°C | Hum: 64.8% | Vib: 0.647g | Power Drift: 1.530